In [20]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import EarlyStopping
from keras import optimizers

In [21]:
import numpy as np
import matplotlib.pyplot as plt

### 데이터 수 확장


In [60]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                    horizontal_flip=True,
                                    width_shift_range=0.1,
                                    height_shift_range=0.1,
                                    rotation_range=30,
                                    vertical_flip=True
                                    )

test_datagen = ImageDataGenerator(rescale=1./255)

In [72]:
train_generator = train_datagen.flow_from_directory('./images/',
                                                    target_size=(48, 48),  # 이미지 크기 조정
                                                    batch_size=2,
                                                    class_mode='categorical',  # 분류 작업인 경우
                                                    shuffle=False  # 데이터를 무작위로 섞을지 여부
                                                    )

Found 2996 images belonging to 3 classes.


In [ ]:
len(train_generator)  # batch_size에 따라서 달라짐

In [26]:
all_images=[]  # feature
all_labels=[]  # target

for _ in range(len(train_generator)):
    batch_images, batch_labels = next(train_generator)
    all_images.extend(batch_images)
    all_labels.extend(batch_labels)

In [27]:
all_labels = np.vstack(all_labels)
all_labels.shape

(2996, 3)

In [38]:
all_images = np.array(all_images)
all_images.shape

(2996, 48, 48, 3)

In [29]:
from sklearn.model_selection import train_test_split

In [30]:
train_X, train_y, test_X, test_y = train_test_split(all_images, all_labels,
                                                    random_state=12,
                                                    train_size=0.8,
                                                    test_size=0.2,
                                                    stratify=all_labels)

In [31]:
test_X

array([[0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       ...,
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.]], dtype=float32)

### VGG16 불러오기 (전이학습)

In [32]:
from keras.applications import VGG16

In [33]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
transfer_model.trainable=False
transfer_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

### DNN 모델 생성

In [42]:
model = Sequential()
model.add(transfer_model)
model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 1, 1, 512)         14714688  
                                                                 
 flatten_2 (Flatten)         (None, 512)               0         
                                                                 
 dense_4 (Dense)             (None, 64)                32832     
                                                                 
 activation_4 (Activation)   (None, 64)                0         
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_5 (Dense)             (None, 3)                 195       
                                                                 
 activation_5 (Activation)   (None, 3)                

In [43]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.001), metrics='accuracy')

In [44]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)

In [45]:
all_images.shape, all_labels.shape

((2996, 48, 48, 3), (2996, 3))

In [59]:
# history = model.fit(all_images, all_labels,
#                     epochs=5,
#                     validation_split=0.2,
#                     callbacks=[early_stopping_callback])

### CNN 모델

In [47]:
from keras.applications import VGG16

In [48]:
transfer_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3))
transfer_model.trainable=False
transfer_model.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, 48, 48, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 48, 48, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 48, 48, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 24, 24, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 24, 24, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 24, 24, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 12, 12, 128)       0     

In [ ]:
# model = Sequential()
# model.add(Conv2D(32, kernel_size=3, input_shape=(48, 48, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(32, kernel_size=3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Conv2D(64, kernel_size=3))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=2))

# model.add(Flatten())
# model.add(Dense(64))
# model.add(Activation('relu'))
# model.add(Dropout(0.5))
# model.add(Dense(1))
# model.add(Activation('softmax'))

In [49]:
model.compile(loss='categorical_crossentropy', optimizer=optimizers.Adam(learning_rate=0.0002), metrics='accuracy')

In [50]:
early_stopping_callback = EarlyStopping(monitor='val_loss', patience=5)